In [1]:
import pandas as pd
import pyterrier as pt
import yaml
import os
from src.load_index import load_index, load_topics, load_qrels, tag
from src.extend_runs import extend_run_full
import sqlite3
from repro_eval.Evaluator import RpdEvaluator
import pytrec_eval
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from argparse import ArgumentParser
from repro_eval.util import arp, arp_scores

if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

/Users/jueri/dev/CLEF2024-LongEval-CIR/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [6]:
index = load_index("t3")
topics = load_topics("t3", "test")
qrels = load_qrels("t3", "test")


19:54:42.954 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
>>> Loaded index with  2049729 documents.


In [8]:
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")
XSqrA_M = pt.BatchRetrieve(index, wmodel="XSqrA_M", verbose=True)
DPH = pt.BatchRetrieve(index, wmodel="DPH", verbose=True)
rm3_pipe = bm25 >> pt.rewrite.RM3(index) >> bm25


In [13]:
pt.Experiment(
    [tf_idf, bm25, pl2, XSqrA_M, DPH, rm3_pipe],
    topics,
    qrels,
    eval_metrics=["P_10", "bpref", "ndcg", "num_rel_ret", "map", "recip_rank"],
    baseline=0,
    verbose=True
)

pt.Experiment:   0%|          | 0/6 [05:43<?, ?system/s]


TypeError: RelevanceEvaluator.__init__() got an unexpected keyword argument 'judged_docs_only_flag'